In [12]:
import numpy as np

def get_signature(cosmic_filepath: str):
    with open(cosmic_filepath) as f:
        lines = f.read().split("\n")[:-1]

    mat = np.array(list(map(lambda x: x.split("\t"), lines)))
    grch38_sig = mat[:, 2][1:].astype(np.float32)

    return grch38_sig

    '''
    print(grch38.shape)
    print(grch38)
    '''

def get_hg38_32vec():
    hg38_32vec = np.repeat(np.load("./hg38-32vec.npy"), 3).astype(np.float32)
    
    return hg38_32vec

def triplicate(vec: np.ndarray):
    return np.repeat(vec, 3)

sbs1 = get_signature("./COSMIC_signatures/v3.3_SBS7d_PROFILE.txt")
sbs5 = get_signature("./COSMIC_signatures/v3.3_SBS5_PROFILE.txt")
hg38_32vec = get_hg38_32vec()

BRCA2_topcatvec = triplicate(np.load("./extracts/2024-08-16-10:39:41.637822_extract/geneTrimerVecs/BRCA2_TopCategory_Trimer_Frequency_Vec.npy"))

mutsig_rates_sbs1 = sbs1/hg38_32vec
mutsig_rates_sbs5 = sbs5/hg38_32vec

sbs1_numerator = np.dot(mutsig_rates_sbs1, BRCA2_topcatvec)
sbs5_numerator = np.dot(mutsig_rates_sbs5, BRCA2_topcatvec)

sbs1_denominator = np.dot(mutsig_rates_sbs1, BRCA2_trimer_freq_vec)
sbs5_denominator = np.dot(mutsig_rates_sbs5, BRCA2_trimer_freq_vec)

BRCA2_sbs1_susceptibility = sbs1_numerator / sbs1_denominator
BRCA2_sbs5_susceptibility = sbs5_numerator / sbs5_denominator

print(BRCA2_sbs1_susceptibility)
print(BRCA2_sbs5_susceptibility)

0.00019998064001824318
0.00018360673870927477


In [5]:
import numpy as np
from CalcMRate.CalcMRate2 import CalcMRate

c = CalcMRate("./bed_data/BRCA2_hg38.bed", "./fasta_data/hg38.fa", "./extraction_dir", "extracted_test", np.random.rand(96))
out = c.get_spec32()
trinlist, BRCA2_trimer_freq_vec = out

GENERATION OF 32-vec DESCRIPTOR | EXTRACTED SEQUENCES


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 12.42it/s]


In [7]:
BRCA2_trimer_freq_vec = np.repeat(np.array(BRCA2_trimer_freq_vec), 3)
BRCA2_trimer_freq_vec.shape

(96,)

In [13]:
out_table = f"""
BRCA2 Susceptibility\t\tSBS\t\tnumerator\t\t\tdenominator
{BRCA2_sbs1_susceptibility}\t\t1\t\t{sbs1_numerator}\t\t{sbs1_denominator}
{BRCA2_sbs5_susceptibility}\t\t5\t\t{sbs5_numerator}\t\t{sbs5_denominator}
"""

print(out_table)


BRCA2 Susceptibility		SBS		numerator			denominator
0.00019998064001824318		1		3.1938670522332594e-08		0.00015970881241013628
0.00018360673870927477		5		2.9492780337717073e-08		0.00016063016284176973

